In [ ]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [ ]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
%azureml history off

In [1]:
%matplotlib inline

import keras.callbacks as cb
from keras.datasets import mnist
from keras.layers.core import Activation, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import SGD,Adam
from keras.regularizers import l1, l2
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor

from matplotlib import pyplot as plt
import numpy as np
import time
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
data = pd.read_csv("matches.csv", header=0, encoding="latin-1")

prepared_data = pd.DataFrame()

homeVenue = list()
for i, d in data["team1Text"].iteritems():
    v = data["venue"].get(i)
    h = False
    if isinstance(v, str) and d in v:
        h = True
    homeVenue.append(h)
    
prepared_data["date"] = data.date
prepared_data["name"] = preprocessing.LabelEncoder().fit_transform(data["team1"])
prepared_data["opponentName"] = preprocessing.LabelEncoder().fit_transform(data["team2"])
prepared_data["homeVenue"] = homeVenue
prepared_data["neutralVenue"] = [not v for v in homeVenue]
prepared_data["homeScore"] = data.team1Score
prepared_data["opponentScore"] = data.team2Score
prepared_data["differenceScore"] = data.team1Score - data.team2Score
prepared_data["homeWin"] = data.team1Score > data.team2Score
prepared_data["opponentWin"] = data.team1Score < data.team2Score
prepared_data["draw"] = data.team1Score == data.team2Score

prepared_data

,date,name,opponentName,homeVenue,neutralVenue,homeScore,opponentScore,differenceScore,homeWin,opponentWin,draw
0,19560930,13,114,True,False,7,0,7,True,False,False
1,19561003,91,54,True,False,2,1,1,True,False,False
2,19561111,68,18,True,False,6,3,3,True,False,False
3,19561205,60,54,True,False,5,2,3,True,False,False
4,19570310,63,179,True,False,2,2,0,False,False,True
5,19570320,139,114,True,False,4,1,3,True,False,False
6,19570425,96,141,True,False,1,0,1,True,False,False
7,19570501,217,189,True,False,1,0,1,True,False,False
8,19570501,143,153,True,False,3,0,3,True,False,False
9,19570501,217,189,True,False,1,0,1,True,False,False


In [80]:
cols = prepared_data.columns
x_columns = cols.drop(["homeScore", "opponentScore"])

x = prepared_data[x_columns]
y = prepared_data[["opponentScore"]]
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [85]:
def define_model():
    model = Sequential()
    
    # First Layer
    model.add(Dense(64, input_dim=9, activation="relu"))
    model.add(Dropout(0.5))
    
    # Second Layer
    model.add(Dense(32, activation="relu"))
    model.add(Dropout(0.5))
    
    # Third Layer
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.5))
    
    # Softmax Layer
    model.add(Dense(1, activation="sigmoid"))
    
    opt = Adam(lr=0.05)
    model.compile(loss="mean_squared_error", optimizer=opt, metrics=["accuracy"])
    
    return model

In [76]:
def train_model():
    batch_size = 128
    epochs = 20
    
    start_time = time.time()
    model = define_model()
    
    print("Start training")
    histroy = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
    print("Training took {0} seconds.".format(time.time() - start_time))
    
    return model, histroy

In [96]:
def train_regression_model():
    start_time = time.time()
    
    print("Start training")
    estimator = KerasRegressor(build_fn=define_model, epochs=20, batch_size=128, verbose=1)
    estimator.fit(X_train, y_train)
    print("Training took {0} seconds.".format(time.time() - start_time))
    
    return estimator

In [104]:
#trained_model, training_history = train_model()
trained_estimator = train_regression_model()
pred = trained_estimator.predict(X_test.head(1))
pred.std()

Start training
Epoch 1/20
25692/25692 [==============================] - 1s 50us/step - loss: 1.9253 - acc: 0.3397
Epoch 2/20
25692/25692 [==============================] - 0s 19us/step - loss: 1.9332 - acc: 0.3379
Epoch 3/20
25692/25692 [==============================] - 0s 19us/step - loss: 1.9298 - acc: 0.3350
Epoch 4/20
25692/25692 [==============================] - 0s 19us/step - loss: 1.9334 - acc: 0.3341
Epoch 5/20
25692/25692 [==============================] - 1s 21us/step - loss: 1.9321 - acc: 0.3389
Epoch 6/20
25692/25692 [==============================] - 0s 19us/step - loss: 1.9284 - acc: 0.3391
Epoch 7/20
25692/25692 [==============================] - 0s 18us/step - loss: 1.9146 - acc: 0.3399
Epoch 8/20
25692/25692 [==============================] - 0s 19us/step - loss: 1.9205 - acc: 0.3371
Epoch 9/20
25692/25692 [==============================] - 0s 19us/step - loss: 1.9199 - acc: 0.3398
Epoch 10/20
25692/25692 [==============================] - 0s 18us/step - loss: 1.917

0.0

In [20]:
def TestModel(model=None, data=None):
    if model is None:
        print("Must provide a trained model.")
        return
    if data is None:
        print("Must provide data.")
        return
    x_test, y_test = data
    scores = model.evaluate(x_test, y_test)
    return scores

In [83]:
test_score = TestModel(model=trained_model, data=[X_test, y_test])
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

8564/8564 [==============================] - 0s 23us/step
Test loss 1.7491, accuracy 32.84%


In [89]:
X_new = X_test.head(1)
print(X_new)
y_new = trained_model.predict_classes(X_new, batch_size=1)
print(y_new)
# print("X=%s, Predicted=%s" % (X_new[0], y_new[0]))
#for i in range(len(X_new)):
#    print("X=%s, Predicted=%s" % (X_new[i], y_new[i]))

          date  name  opponentName homeVenue neutralVenue  differenceScore  \
7151  19871014    69           183     False         True                0   

     homeWin opponentWin  draw  
7151   False       False  True  
[[0]]
